# Import libraries

In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Read dataset

In [2]:
diamonds_test = pd.read_csv('data/diamonds.csv')
diamonds_rick = pd.read_csv('data/rick_diamonds.csv')

# Data Cleaning 

# Exploratory Analysis

In [10]:
diamonds_test.corr()['price']

carat    0.922186
depth   -0.012037
table    0.127832
price    1.000000
x        0.885019
y        0.864059
z        0.860247
Name: price, dtype: float64

# Data transformation

# Modeling

In [18]:
X = diamonds_test[['carat']]
y = diamonds_test['price']

model = LinearRegression()
model.fit(X,y)

price_pred = model.predict(diamonds_rick[['carat']])
diamonds_rick['price_predicted'] = price_pred

# To CSV

In [19]:
diamonds_rick.to_csv('testes_price/price_1model', index=False)
diamonds_rick

,carat,cut,color,clarity,depth,table,x,y,z,price_predicted
0,0.91,Very Good,H,SI2,63.4,57.0,6.10,6.18,3.89,4804.879333
1,0.38,Good,F,VS2,60.3,61.0,4.68,4.71,2.83,692.422362
2,0.52,Ideal,H,VS2,61.8,56.0,5.19,5.16,3.20,1778.731751
3,1.10,Ideal,G,VS1,61.3,56.0,6.65,6.69,4.09,6279.156361
4,2.28,Ideal,G,SI2,61.6,57.0,8.44,8.37,5.18,15435.192637
...,...,...,...,...,...,...,...,...,...,...
4995,0.30,Very Good,D,SI2,62.2,59.0,4.25,4.30,2.66,71.674140
4996,0.51,Ideal,F,VVS2,61.9,56.0,5.09,5.14,3.16,1701.138223
4997,1.01,Very Good,F,SI1,60.6,60.0,6.37,6.43,3.88,5580.814611
4998,0.31,Ideal,F,VVS1,62.1,56.0,4.35,4.38,2.71,149.267668


# Results


In [ ]:
# erro usando a média foi 3980.71
# erro usando a mediana foi 4255.53
# erro usando a primeira modelagem foi 